### Import Libraries

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() 

from langchain_community.document_loaders import CSVLoader
# embedding 
from langchain.embeddings import OllamaEmbeddings
# vector db 
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

In [ ]:
GEMINI_API_KEY =os.getenv('GEMINI_API_KEY')
GROQ_API_KEY= os.getenv('GROQ_API_KEY')

In [3]:
GROQ_API_KEY

'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'

## Load Data

In [4]:
file_path =r"..\Data\data.csv"
loader =CSVLoader(file_path)
fit_docs =loader.load()
print(fit_docs[0].page_content)

id: 0
exercise_name: Push-Ups
type_of_activity: Strength
type_of_equipment: Bodyweight
body_part: Upper Body
type: Push
muscle_groups_activated: Pectorals, Triceps, Deltoids
instructions: Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.


In [3]:
embeddings =OllamaEmbeddings(model ='plutonioumguy/bge-m3')
# vectore_store =Chroma.from_documents(
#     embedding=embeddings,
#     persist_directory='./chromadb',
#     documents =fit_docs,
#     collection_name='fitness_data'
# )

# # 3. Save the vector store
# vectore_store.persist()

C:\Users\elkhwaga\AppData\Local\Temp\ipykernel_1604\3097276066.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings =OllamaEmbeddings(model ='plutonioumguy/bge-m3')


In [4]:
# # loading chromadb from disk later
vectore_store = Chroma(
    embedding_function=embeddings,
    persist_directory='./chromadb',
    collection_name='fitness_data'
)

C:\Users\elkhwaga\AppData\Local\Temp\ipykernel_1604\3125965979.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectore_store = Chroma(


In [5]:
retriver =vectore_store.as_retriever(search_kwargs={'k':10})

In [8]:
# rag from documents
query = "Which muscles do push-ups work?"
response = retriver.get_relevant_documents(query)
response

C:\Users\elkhwaga\AppData\Local\Temp\ipykernel_14052\1924855797.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = retriver.get_relevant_documents(query)


[Document(metadata={'row': 148, 'source': '..\\Data\\data.csv'}, page_content='id: 148\nexercise_name: Spider Push-Ups\ntype_of_activity: Strength\ntype_of_equipment: Bodyweight\nbody_part: Upper Body\ntype: Push\nmuscle_groups_activated: Pectorals, Triceps, Deltoids, Obliques\ninstructions: Start in a high plank position. As you lower your body, bring one knee towards your elbow, then push back up and switch sides.'),
 Document(metadata={'row': 0, 'source': '..\\Data\\data.csv'}, page_content='id: 0\nexercise_name: Push-Ups\ntype_of_activity: Strength\ntype_of_equipment: Bodyweight\nbody_part: Upper Body\ntype: Push\nmuscle_groups_activated: Pectorals, Triceps, Deltoids\ninstructions: Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'),
 Document(metadata={'row': 48, 'source': '..\\Data\\data.csv'}, page_content='id: 48\nexercise_name: Incline Push-Ups\ntype_of_activity

### RAG Flow

In [6]:
# llm =ChatGoogleGenerativeAI(
#     model ='models/gemini-2.0-flash-lite',
#     temperature=0.4,
#     google_api_key=GOOGLE_API_KEY 
# )

lama_llm =ChatGroq(
    model ='llama3-8b-8192',
    temperature=0.3,
    groq_api_key =GROQ_API_KEY
)

prompt_template ='\n'.join([
    "you are a fitness instructor",
    "answer the question based on the given context about exercises",
    "use only the facts from the context,when answering the question",
    "reply with a structured way,",
    "you must no reply with id number in your result",
    "your reply must have short detials",
    "Question:{question}",
    "context:{context}"
])

prompt =ChatPromptTemplate.from_template(prompt_template)

# rag chain 
fitness_chain= (
    {"question":itemgetter('question'),
     "context":lambda x:retriver.invoke(x['question'])}
    |prompt
    |lama_llm
)


In [ ]:
response = fitness_chain.invoke({"question": "Which exercise targets the core muscles?"})
print(response.content)  
print(response)

The exercises that target the core muscles are:

* Mountain Climbers: This exercise targets the Rectus Abdominis and Hip Flexors.
* Reverse Crunch: This exercise targets the Rectus Abdominis and Hip Flexors.
* Toe Touches: This exercise targets the Rectus Abdominis and Obliques.
* Ab Wheel Rollout: This exercise targets the Rectus Abdominis and Hip Flexors.

These exercises engage the core muscles, including the rectus abdominis, hip flexors, and obliques, to improve overall core strength and stability.


### RAG Evaluation 
__LLM As a Judge__

In [7]:
import pandas as pd
df_question =pd.read_csv('../Data/ground-truth-retrieval.csv')
df_question.head()

,id,question
0,0,What is the starting position for doing push-ups?
1,0,Which muscle groups are activated during push-...
2,0,How do you know when to push back up while doi...
3,0,Do you need any equipment to perform push-ups?
4,0,What part of the body do push-ups primarily ta...


In [8]:
ground_truth =df_question.to_dict(orient='records')
ground_truth[0]

{'id': 0, 'question': 'What is the starting position for doing push-ups?'}

In [ ]:
llm_Judge=ChatGoogleGenerativeAI(
    model ='models/gemini-2.0-flash',
    temperature=0.2,
    google_api_key=GEMINI_API_KEY 
)

Judge_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [10]:
Judge_prompt =ChatPromptTemplate.from_template(Judge_template)

Judge_chain=(
    Judge_prompt
    |llm_Judge
)

In [ ]:
first_recored =ground_truth[0]
answer_llm =fitness_chain.invoke({'question':first_recored['question']})
# print(answer_llm)
response =Judge_chain.invoke({
                'question':first_recored['question'],
                'answer_llm':answer_llm.content
                })

print(response.content)


In [52]:
type(response.content)

str

In [53]:
df_sample = df_question.sample(n=100, random_state=1)

In [54]:
sample = df_sample.to_dict(orient='records')

In [ ]:
from tqdm.auto import tqdm
import json
import time
from json_repair import repair_json 

evaluations = []
for record in tqdm(sample):
    question = record['question']
    answer_llm = answer_llm =fitness_chain.invoke({'question':question}).content

    response =Judge_chain.invoke({
                'question':question,
                'answer_llm':answer_llm
                }).content
    print(response)
    time.sleep(2)
    res_str =repair_json(response)
    evaluation = json.loads(res_str)
    evaluations.append((record, answer_llm, evaluation))

In [57]:
evaluations[0]

({'id': 171,
  'question': 'What is the primary muscle group activated during the Banded Pull-Up?'},
 'The primary muscle group activated during the Banded Pull-Up is the Latissimus Dorsi and Biceps.',
 {'Relevance': 'RELEVANT',
  'Explanation': 'The answer correctly identifies the primary muscle groups activated during a banded pull-up, which are the Latissimus Dorsi and Biceps. Therefore, the answer is relevant to the question.'})

In [66]:
df_eval_ =pd.DataFrame(evaluations,columns=['record','answer','evaluation'])
df_eval_.head()

,record,answer,evaluation
0,"{'id': 171, 'question': 'What is the primary m...",The primary muscle group activated during the ...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
1,"{'id': 115, 'question': 'Can jumping squats be...",Jumping squats can be performed without any eq...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
2,"{'id': 53, 'question': 'Can you explain the pr...","To execute a Dumbbell Lateral Raise, follow th...","{'Relevance': 'PARTLY_RELEVANT', 'Explanation'..."
3,"{'id': 198, 'question': 'How do I ensure my ar...",To ensure your arms are fully extended during ...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
4,"{'id': 19, 'question': 'What part of the body ...","The exercise ""Goblet Squats"" primarily engages...","{'Relevance': 'RELEVANT', 'Explanation': 'The ..."


In [67]:
df_eval_['id'] = df_eval_.record.apply(lambda d: d['id'])
df_eval_['question'] = df_eval_.record.apply(lambda d: d['question'])

df_eval_['relevance'] = df_eval_.evaluation.apply(lambda d: d['Relevance'])
df_eval_['explanation'] = df_eval_.evaluation.apply(lambda d: d['Explanation'])

del df_eval_['record']
del df_eval_['evaluation']

In [68]:
df_eval_.head()

,answer,id,question,relevance,explanation
0,The primary muscle group activated during the ...,171,What is the primary muscle group activated dur...,RELEVANT,The answer correctly identifies the primary mu...
1,Jumping squats can be performed without any eq...,115,Can jumping squats be performed without any eq...,RELEVANT,The answer directly and accurately answers the...
2,"To execute a Dumbbell Lateral Raise, follow th...",53,Can you explain the proper technique for execu...,PARTLY_RELEVANT,The answer provides a basic description of the...
3,To ensure your arms are fully extended during ...,198,How do I ensure my arms are fully extended dur...,RELEVANT,The answer directly addresses the question of ...
4,"The exercise ""Goblet Squats"" primarily engages...",19,What part of the body is primarily engaged in ...,RELEVANT,The answer directly addresses the question by ...


In [69]:
df_eval_.relevance.value_counts()

relevance
RELEVANT           73
PARTLY_RELEVANT    25
NON_RELEVANT        2
Name: count, dtype: int64

In [71]:
df_eval_.to_csv('../Data/RAG_Evaluation_Result.csv',index=False)